In [2]:
import pandas as pd
import numpy as np
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import scipy.ndimage
import random
import re

# WORD: COUSIN

In [13]:
frames_folder_path = "D:\\Datasets\\minor2_dataset2\\3_word_dataset\\cousin"
frames_folder_dir = os.listdir(frames_folder_path)

save = "D:\\Datasets\\minor2_dataset2\\3_word_aug_dataset\\cousin" 

In [4]:
def sort_key(filename):
        match = re.compile(r'^(\d+)\.jpg$').match(filename)
        if match:
            return int(match.group(1))
        else:
            return 0

## Augmentation technique - Rotation 

In [5]:
def rotation(frames, save_path):
    """
    Apply a rotation to a list of frames and save them to a specified directory.
    
    Args:
    frames: A list of frames to be rotated.
    save_path: The path to the directory where the augmented frames should be saved.
    """
    # Define the rotation angle in degrees
    rotation_angle = 10

    # Apply the rotation to all frames
    for i, frame in enumerate(frames):
        # Get the image dimensions
        height, width = frame.shape[:2]
        
        # Calculate the center of the image
        center = (width / 2, height / 2)
        
        # Perform the rotation
        rotation_matrix = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)
        augmented_frame = cv2.warpAffine(frame, rotation_matrix, (width, height))
        
        # Construct the full file path for the augmented frame
        file_path = os.path.join(save_path, f"frame_{i+1}.jpg")
        
        # Save the augmented frame
        cv2.imwrite(file_path, augmented_frame)

In [8]:
for video in frames_folder_dir:
    video_path = os.path.join(frames_folder_path, video,"_5")
    frm_dir = sorted(os.listdir(video_path), key=sort_key)
    new_path = os.path.join(save, video, "_0")
    os.makedirs(new_path, exist_ok=True)
    # List to store frames
    frames = []
    # List all files in the folder
    for filename in frm_dir:
        # Construct the full file path
        file_path = os.path.join(video_path, filename)
        # Read the image file
        frame = cv2.imread(file_path)

        # Append the frame to the list
        frames.append(frame)
        # Now, 'frames' is a list of all frames in the folder
    rotation(frames, new_path)

## Augmentation technique - Horizontal flip

In [11]:
def horizontal_flip(frames, save_path):
    """
    Apply a horizontal flip to a list of frames and save them to a specified directory.
    
    Args:
    frames: A list of frames to be flipped.
    save_path: The path to the directory where the augmented frames should be saved.
    """
    # Apply the horizontal flip to all frames
    for i, frame in enumerate(frames):
        if frame is None:
            print(f"Failed to read frame {i+1}. Skipping.")
            continue
        
        # Flip the frame horizontally
        flipped_frame = cv2.flip(frame, 1)
        
        # Construct the full file path for the flipped frame
        file_path = os.path.join(save_path, f"frame_{i+1}.jpg")
        
        # Save the flipped frame
        cv2.imwrite(file_path, flipped_frame)

In [12]:
for video in frames_folder_dir:
    video_path = os.path.join(frames_folder_path, video,"_5")
    frm_dir = sorted(os.listdir(video_path), key=sort_key)
    new_path = os.path.join(save, video, "_1")
    os.makedirs(new_path, exist_ok=True)
    # List to store frames
    frames = []
    # List all files in the folder
    for filename in frm_dir:
        # Construct the full file path
        file_path = os.path.join(video_path, filename)
        # Read the image file
        frame = cv2.imread(file_path)

        # Append the frame to the list
        frames.append(frame)
        # Now, 'frames' is a list of all frames in the folder
    horizontal_flip(frames, new_path)

## Augmentation technique - brightness, contrast

In [20]:
def adjust_brightness_contrast(frames, save_path, alpha=0.7, beta=-10):
    """
    Apply brightness and contrast adjustments to a list of frames and save them to a specified directory.
    
    Args:
    frames: A list of frames to be adjusted.
    save_path: The path to the directory where the augmented frames should be saved.
    alpha: Contrast control (1.0-3.0). Default is 0.7.
    beta: Brightness control (0-100). Default is -10.
    """
    # Apply the brightness and contrast adjustment to all frames
    for i, frame in enumerate(frames):
        if frame is None:
            print(f"Failed to read frame {i+1}. Skipping.")
            continue
        
        # Adjust the brightness and contrast
        adjusted_frame = cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)
        
        # Construct the full file path for the adjusted frame
        file_path = os.path.join(save_path, f"frame_{i+1}.jpg")
        
        # Save the adjusted frame
        cv2.imwrite(file_path, adjusted_frame)

In [21]:
for video in frames_folder_dir:
    video_path = os.path.join(frames_folder_path, video,"_5")
    frm_dir = sorted(os.listdir(video_path), key=sort_key)
    new_path = os.path.join(save, video, "_2")
    os.makedirs(new_path, exist_ok=True)
    # List to store frames
    frames = []
    # List all files in the folder
    for filename in frm_dir:
        # Construct the full file path
        file_path = os.path.join(video_path, filename)
        # Read the image file
        frame = cv2.imread(file_path)

        # Append the frame to the list
        frames.append(frame)
        # Now, 'frames' is a list of all frames in the folder
    adjust_brightness_contrast(frames, new_path)

## Augmentation technique - random erase

In [22]:
def random_erase(frame, p=0.5, s=(0.02, 0.45), r=(0.3, 4)):
    if random.random() < p:
        h, w = frame.shape[:2]
        s_h, s_w = int(h * s[0]), int(w * s[1]) # Convert to integers
        r_h, r_w = int(s_h * r[0]), int(s_w * r[1])
        x = random.randrange(0, h - s_h)
        y = random.randrange(0, w - s_w)
        frame[y:y+r_h, x:x+r_w] = 0 # Set the selected area to black
    return frame


In [23]:
def apply_random_erase(frames, save_path):
    """
    Apply random erasing to a list of frames and save them to a specified directory.
    
    Args:
    frames: A list of frames to be erased.
    save_path: The path to the directory where the augmented frames should be saved.
    """
    # Apply the random erase to all frames
    for i, frame in enumerate(frames):
        if frame is None:
            print(f"Failed to read frame {i+1}. Skipping.")
            continue
        
        # Apply the random erase
        augmented_frame = random_erase(frame)
        
        # Construct the full file path for the erased frame
        file_path = os.path.join(save_path, f"frame_{i+1}.jpg")
        
        # Save the erased frame
        cv2.imwrite(file_path, augmented_frame)

In [24]:
for video in frames_folder_dir:
    video_path = os.path.join(frames_folder_path, video,"_5")
    frm_dir = sorted(os.listdir(video_path), key=sort_key)
    new_path = os.path.join(save, video, "_3")
    os.makedirs(new_path, exist_ok=True)
    # List to store frames
    frames = []
    # List all files in the folder
    for filename in frm_dir:
        # Construct the full file path
        file_path = os.path.join(video_path, filename)
        # Read the image file
        frame = cv2.imread(file_path)

        # Append the frame to the list
        frames.append(frame)
        # Now, 'frames' is a list of all frames in the folder
    apply_random_erase(frames, new_path)

## Augmentation technique - noise

In [25]:
def apply_gaussian_noise(frames, save_path, mean=0, std_dev=1):
    """
    Apply Gaussian noise to a list of frames and save them to a specified directory.
    
    Args:
    frames: A list of frames to be augmented with Gaussian noise.
    save_path: The path to the directory where the augmented frames should be saved.
    mean: Mean of the Gaussian distribution. Default is 0.
    std_dev: Standard deviation of the Gaussian distribution. Default is 1.
    """
    # Apply Gaussian noise to all frames
    for i, frame in enumerate(frames):
        if frame is None:
            print(f"Failed to read frame {i+1}. Skipping.")
            continue
        
        # Add Gaussian noise
        noise = np.random.normal(mean, std_dev, frame.shape).astype(np.uint8)
        noisy_frame = cv2.add(frame, noise)
        
        # Ensure the pixel values are within the valid range [0, 255]
        noisy_frame = np.clip(noisy_frame, 0, 255).astype(np.uint8)
        
        # Construct the full file path for the noisy frame
        file_path = os.path.join(save_path, f"frame_{i+1}.jpg")
        
        # Save the noisy frame
        cv2.imwrite(file_path, noisy_frame)

In [27]:
for video in frames_folder_dir:
    video_path = os.path.join(frames_folder_path, video,"_5")
    frm_dir = sorted(os.listdir(video_path), key=sort_key)
    new_path = os.path.join(save, video, "_4")
    os.makedirs(new_path, exist_ok=True)
    # List to store frames
    frames = []
    # List all files in the folder
    for filename in frm_dir:
        # Construct the full file path
        file_path = os.path.join(video_path, filename)
        # Read the image file
        frame = cv2.imread(file_path)

        # Append the frame to the list
        frames.append(frame)
        # Now, 'frames' is a list of all frames in the folder
    apply_gaussian_noise(frames, new_path)

### # Geometric transformations

In [5]:
# # Initialize the ImageDataGenerator with geometric transformations
# datagen = ImageDataGenerator(
#     rotation_range=20, # Rotate the image by up to 20 degrees
#     width_shift_range=0.2, # Shift the image horizontally by up to 20% of the width
#     height_shift_range=0.2, # Shift the image vertically by up to 20% of the height
#     horizontal_flip=True, # Flip the image horizontally
#     vertical_flip=True, # Flip the image vertically
#     zoom_range=0.2, # Zoom in on the image by up to 20%
#     fill_mode='nearest' # Fill in the gaps with the nearest pixel
# )

In [6]:
# def frame_transformation(frames, out_video):
#     count = 0
#     # Apply the generator to the frames
#     for frame in frames: # Assuming 'frames' is a list of video frames
#         # Add an extra dimension to represent the batch size
#         frame_with_batch_dim = np.expand_dims(frame, axis=0)

#         # Now, frame_with_batch_dim has shape (1, height, width, channels)
#         augmented_frame = next(datagen.flow(frame_with_batch_dim, batch_size=1))[0]
#         image_path = os.path.join(out_video, "{}.jpg".format(count))
#         cv2.imwrite(image_path, augmented_frame)
#         count += 1

In [7]:
# for video in frames_folder_dir:
#     video_path = os.path.join(frames_folder_path, video)
#     frm_dir = sorted(os.listdir(video_path), key=sort_key)
#     new_path = os.path.join(save, video, "_0")
#     os.makedirs(new_path, exist_ok=True)
#     # List to store frames
#     frames = []
#     # List all files in the folder
#     for filename in frm_dir:
#         # Construct the full file path
#         file_path = os.path.join(video_path, filename)
#         # Read the image file
#         frame = cv2.imread(file_path)

#         # Append the frame to the list
#         frames.append(frame)
#         # Now, 'frames' is a list of all frames in the folder
#     frame_transformation(frames, new_path)

### Color Space transformations

In [8]:
# # Initialize the ImageDataGenerator with color space transformations
# datagen = ImageDataGenerator(
#     brightness_range=[0.2, 1.0], # Intensify the brightness of the image
#     channel_shift_range=20.0, # Shift the color channels by up to 20
#     fill_mode='nearest' # Fill in the gaps with the nearest pixel
# )

In [9]:
# def col_trans(frames, out_video):
#     count = 0
#     # Apply the generator to the frames
#     for frame in frames: # Assuming 'frames' is a list of video frames
#         # Add an extra dimension to represent the batch size
#         frame_with_batch_dim = np.expand_dims(frame, axis=0)

#         # Now, frame_with_batch_dim has shape (1, height, width, channels)
#         augmented_frame = next(datagen.flow(frame_with_batch_dim, batch_size=1))[0]
#         image_path = os.path.join(out_video, "{}.jpg".format(count))
#         cv2.imwrite(image_path, augmented_frame)
#         count += 1      

In [10]:
# for video in frames_folder_dir:
#     video_path = os.path.join(frames_folder_path, video)
#     frm_dir = sorted(os.listdir(video_path), key=sort_key)
#     new_path = os.path.join(save, video, "_1")
#     os.makedirs(new_path, exist_ok=True)
#     # List to store frames
#     frames = []
#     # List all files in the folder
#     for filename in frm_dir:
#         # Construct the full file path
#         file_path = os.path.join(video_path, filename)
#         # Read the image file
#         frame = cv2.imread(file_path)

#         # Append the frame to the list
#         frames.append(frame)
#         # Now, 'frames' is a list of all frames in the folder
#     col_trans(frames, new_path)

### Sharpening Transformation

In [11]:
# def apply_kernel(frames,out_video):
#     count = 0
#     kernel = np.array([[-1, -1, -1],
#                        [-1, 9, -1],
#                        [-1, -1, -1]])
#     for frame in frames:
#         cv2.filter2D(frame, -1, kernel)
#         image_path = os.path.join(out_video, "{}.jpg".format(count))
#         cv2.imwrite(image_path, frame)
#         count += 1

In [12]:
# for video in frames_folder_dir:
#     video_path = os.path.join(frames_folder_path, video)
#     frm_dir = sorted(os.listdir(video_path), key=sort_key)
#     new_path = os.path.join(save, video, "_2")
#     os.makedirs(new_path, exist_ok=True)
#     # List to store frames
#     frames = []
#     # List all files in the folder
#     for filename in frm_dir:
#         # Construct the full file path
#         file_path = os.path.join(video_path, filename)
#         # Read the image file
#         frame = cv2.imread(file_path)

#         # Append the frame to the list
#         frames.append(frame)
#         # Now, 'frames' is a list of all frames in the folder
#     apply_kernel(frames, new_path)

### Random Erase

In [13]:
# def random_erase(frames, out_video, p=0.5, s=(0.03, 0.4), r=(0.3, 3)):
#     count = 0
#     for frame in frames:
#         if random.random() < p:
#             h, w = frame.shape[:2]
#             s_h, s_w = int(h * s[0]), int(w * s[1]) # Convert to integers
#             r_h, r_w = int(s_h * r[0]), int(s_w * r[1])
#             x = random.randrange(0, h - s_h)
#             y = random.randrange(0, w - s_w)
#             frame[y:y+r_h, x:x+r_w] = 0 # Set the selected area to black
#             image_path = os.path.join(out_video, "{}.jpg".format(count))
#             cv2.imwrite(image_path, frame)
#             count += 1

In [14]:
# for video in frames_folder_dir:
#     video_path = os.path.join(frames_folder_path, video)
#     frm_dir = sorted(os.listdir(video_path), key=sort_key)
#     new_path = os.path.join(save, video, "_3")
#     os.makedirs(new_path, exist_ok=True)
#     # List to store frames
#     frames = []
#     # List all files in the folder
#     for filename in frm_dir:
#         # Construct the full file path
#         file_path = os.path.join(video_path, filename)
#         # Read the image file
#         frame = cv2.imread(file_path)

#         # Append the frame to the list
#         frames.append(frame)
#         # Now, 'frames' is a list of all frames in the folder
#     random_erase(frames, new_path)

### adding noise 

In [15]:
# def noise(frames, out_video):
#     count = 0
#     for frame in frames:
#         frame = cv2.add(frame, np.random.randint(0, 60, frame.shape, dtype=np.uint8))
#         image_path = os.path.join(out_video, "{}.jpg".format(count))
#         cv2.imwrite(image_path, frame)
#         count += 1

In [16]:
# # Specify the path to the folder containing the frames
# for video in frames_folder_dir:
#     video_path = os.path.join(frames_folder_path, video)
#     frm_dir = sorted(os.listdir(video_path), key=sort_key)
#     new_path = os.path.join(save, video, "_4")
#     os.makedirs(new_path, exist_ok=True)
#     # List to store frames
#     frames = []
#     # List all files in the folder
#     for filename in frm_dir:
#         # Construct the full file path
#         file_path = os.path.join(video_path, filename)
#         # Read the image file
#         frame = cv2.imread(file_path)

#         # Append the frame to the list
#         frames.append(frame)
#         # Now, 'frames' is a list of all frames in the folder
#     noise(frames, new_path)